## LOAD PACKAGES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from IPython.display import Image  
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import plotly.express as px
import time
from scipy import stats
from scipy.stats import zscore
import os
import statistics
from statistics import *

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn import neighbors
import time
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import math

## LOAD FUNCTIONS

In [ ]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder

class OneHotEncoder:
    
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
         OneHotEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = OneHotEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = OneHotEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [2]:
import numpy as npa
from scipy import stats
from scipy.stats import zscore

#detetction only on numeric data so need to encode if we want on all the data

def pre_detect_outlier(df, threshold):

    outlier_i = []
    outlier_j=[]
    outlier_var=[]

    numeric_cols = df.select_dtypes(include=[np.number]).columns #'int','float', 'cfloat'
    #print(numeric_cols)
    if len(numeric_cols)>0: 
        data_zscore =df[numeric_cols].apply(stats.zscore)
        data_zscore = data_zscore.abs()

            #outliers preprocessing numeric
        for var in data_zscore.columns: #name of the var, here only numeric
            var_out=0
            for i in range(data_zscore.shape[0]):
                #print(i)
                if data_zscore.loc[i,var] > threshold:
                    var_out=var_out+1
                    outlier_i.append(i)
                    outlier_j.append(list(data_zscore.columns).index(var))
                    if var_out> data_zscore.shape[0]/2:
                        outlier_var.append(var)


    data_cat = df.select_dtypes(include=['object']).copy() #, "category"


    #outliers preprocessing categoric
    for var in data_cat.columns: #name of the var, here only cat
        #print(var)
        var_out=0
        #pour les modèles, moins que 100
        dict_var= pd.DataFrame.from_dict(data_cat[var].value_counts()).reset_index()
        dict_var=dict_var.rename(columns={'attribut': var, var:'count'})
        dict_var['count']=dict_var['count']/df.shape[0]
        #print(dict_var)
        torename=list(dict_var[(dict_var['count']!=1/df.shape[0]) & (dict_var['count']==min(dict_var['count']))]['index'])
        #print(torename)
        df[var].replace(torename, dict_var['index'][0])


    return(outlier_i, outlier_j, outlier_var)

In [ ]:
def remove_outliers(df,outlier_index,outlier_var):
    if outlier_var:
        df=df.drop(outlier_var,inplace=True, axis=1) #columns
    if outlier_index:
        df=df.drop(outlier_index, axis=0) #rows
        
    return df

In [ ]:
def clean(df): #simple data cleaning
    df= df.dropna(axis = 1, how = ‘any’) #If any NA values are present, drop that row or column.   
    df = df.loc[:,df.apply(pd.Series.nunique) != 1] #drop cst columns
    df=df.dropna(axis=0, how=‘any’) #same for row
    #df=df.dropna(axis=1, how=‘all’) #if all values are NA drop that column
    #df=df.dropna(axis=0, how=‘all’) #same for row
    return df

In [ ]:
#generation of random outliers with differents rate

def generate_outliers_multi(df_or): 
    
    from random import choices
    from random import choice
    from random import uniform
    from sklearn.utils import shuffle

    #generation of random outliers with differents rate


    epsilon_ratio=[0.01,0.03,0.1,0.15] #to be sure to have at least 1 outlier generated
    epsilon=choice(epsilon_ratio)
    index_out=[]
    var_out=[]

    #initialisation 
    df_or=df_or.reset_index()
    outlier_size=int(abs(epsilon*df_or.shape[0]))
    if outlier_size==0:
        outlier_size=1
    data_outliers=pd.DataFrame()
    df_or_index=[1] * df_or.shape[0] #1 for normal -1 for outlier
    liste_string=['blanc', 'noir', 'pourpre', 'rouge', 'orange', 'jaune', 'vert', 'bleu', 'violet', 'ivoire', 'crème', 'beige', 'rose', 'kaki', 'brun', 'marron', 'bordeaux']
    matrix_outliers=np.zeros([df_or.shape[0],df_or.shape[1]])

    for i_out in range(outlier_size):
        #print(i_out)
        #le nombre de outliers selon ratio

        outliers= df_or.sample(n = 1) #take only one of the row
        #print(outliers)
        index_out=outliers.index.tolist()
        #df_or=df_or.drop(index_out) #removing the element selected, not removing but transformed
        #print(index_out)
        #print(df_or_index)
        df_or_index[index_out[0]]=-1  #outlier label changed
        #print(df_or_index)


        for var_j in range(outliers.shape[1]): #var_j is the index
            

            if choices(['do','do_not'], weights=[0.6, 0.4])=='do': #bruit pour faire outliers on some variables or not

                #si il y a un changement on le stocke dans la matrix

                matrix_outliers[index_out[0]][var_j]=1

                         #separate the outliers depending of the type of object directly
                if type(outliers.iloc[0,var_j])==str: #si on a affaire à du catégorique

                    df_or.iloc[index_out[0],var_j]= choice(liste_string) #generate random from a string liste

                else: #si c'est du numérique
                    cst=uniform(2,np.max(df_or[df_or.columns[var_j]])) #generate random constant

                    noise=np.random.normal(0,1,1) #mean, std and nbr of elmts
                    df_or.iloc[index_out[0],var_j]=outliers.iloc[0,var_j]*cst + noise

        #data_outliers=pd.concat([data_outliers,outliers]) #to have all the outlier_size outliers



            #outliers_index=['outliers'] * outlier_size
    #outliers_df = pd.DataFrame(data=data_outliers,columns = df_or.columns)

    #df = pd.concat([df_or, outliers_df]) #to have the dataset + the generated outliers

        #print(len(df_or_index))
        #print(df.shape)
    df=df_or
    df['label_outlier']=df_or_index

    df.reset_index()
    df,matrix_outliers=shuffle(df,matrix_outliers) #shuffle the data and the matrix the same way



    return(df,matrix_outliers, epsilon)

In [ ]:
def confusion_matrix_for_matrix(matrix_out, algo_matrix): 
    #the matrix need to be of the same shape

    TP=0
    TN=0
    FN=0
    FP=0

    matrix_out=np.matrix(matrix_out)
    algo_matrix=np.matrix(algo_matrix)

    for var in range(matrix_out.shape[1]):
        for elmt in range(matrix_out.shape[0]):
            if (matrix_out[elmt, var]==0) and (algo_matrix[elmt, var]==0): #correct
                TP=TP+1
            if (matrix_out[elmt,var]==1) and (algo_matrix[elmt, var]==1): #correct
                TN=TN+1
            if (matrix_out[elmt, var]==1) and (algo_matrix[elmt, var]==0): #no correct
                FP=FP+1
            if (matrix_out[elmt, var]==0) and (algo_matrix[elmt, var]==1): #no correct
                FN=FN+1

    #confusion_matrix=np.matrix([[TP, FP], [FN, TN]])
    ACC= (TP+TN)/ (TP+TN+FP+FN)
    #Recall=TP/(TP+FN)
    #Precision=TP/(TP+FP)
    F1=2*TP/(2*TP+FP+FN)
    
    return TP, FN, FP,TN, ACC, F1

In [ ]:
def isolationforrest_uni(df): #à reprendre pour la partie final score
    #
    # theta between 0 and 1 to decide a global outlier
    #
    #encode categorical into multiple code
    #df=multicolumnlabelencoder(df)
    
    #obj_df = df.select_dtypes(include=['object','category','datetime64','bool']).copy()
    #df=class_all.MultiColumnLabelEncoder(columns = obj_df.columns).fit_transform(df)
    #df=OneHotEncoder(columns=obj_df.columns).fit_transform(df)

    #score of isolation forrest
    iforest=pd.DataFrame()
    for columns in df.columns:
        #print(columns)
        model=IsolationForest(n_estimators=90, contamination=0.2)
        model.fit(df[[columns]])
        iforest['anomaly_'+columns ]=model.predict(df[[columns]])
        
    iforest=iforest.replace(1,0)
    iforest=iforest.replace(-1, 1)

    return(iforest) #return the matrix for 0 no outliers attributes and 1 outlier attribute

In [ ]:
#https://betterprogramming.pub/anomaly-detection-with-isolation-forest-e41f1f55cc6

def isolationforrest_multi(df): #, threshold):  
    #
    # theta between 0 and 1 to decide a global outlier
    #
    #encode categorical into multiple code
    #obj_df = df.select_dtypes(include=['object','category','datetime64','bool']).copy()
    #df=multicolumnlabelencoder(df)
    
    #ALTERNATIVE the results are awfull
   # iforest=pd.DataFrame()
   # for columns in df.columns:
        #print(columns)
       # model=IsolationForest(n_estimators=50, max_samples='auto', contamination='auto',max_features=1.0)
      #  model.fit(df[[columns]])
      #  iforest['anomaly_'+columns ]=model.predict(df[[columns]])
        
    #iforest['Final_score']='normal' #normal (encode 1 for normal and -1 for outliers)
   # for obs in range(iforest.shape[0]):
      #  score_obs=0
       # for columns in df.columns:
         #   if iforest.loc[obs,'anomaly_'+columns ]==-1:
           #     score_obs=score_obs+1
      #  if score_obs/df.shape[1] > threshold: #% de variables déclarées outliers
        #    iforest.loc[obs,'Final_score']='outliers'
        #print(score_obs/df.shape[1])
   # df['label_outliers']= 1
   # df.loc[np.array(iforest['Final_score'].values)=='outliers', 'label_outliers']=-1

    
    
    iforest=IsolationForest(n_estimators=90, contamination=0.2)
    pred= iforest.fit_predict(df)
    df['scores']=iforest.decision_function(df)
    df['anomaly_label']=pred
    #in this case iforest_df['anomaly_label']
   
    return df #, threshold

In [ ]:
def metrics(known_outliers,predict_outliers):
    acc=accuracy_score(known_outliers, predict_outliers)
    #fpr, tpr, thresholds = metrics.roc_curve(known_outliers, predict_outliers, pos_label=2)
    #auc=metrics.auc(fpr, tpr)
    f1=f1_score(known_outliers, predict_outliers, average=None)
    

    return acc,f1   #auc if needed to be added but problem in the code

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

def LOF_b(df,n_neighbors,contamination):    
    
    #encode categorical into multiple code
    #df=multicolumnlabelencoder(df)
    #obj_df = df.select_dtypes(include=['object']).copy()
    #df=MultiColumnLabelEncoder(columns = obj_df.columns).fit_transform(df)

    # fit the model for outlier detection (default)
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    y_pred=lof.fit_predict(df)

    lof_scores = lof.negative_outlier_factor_ #outlier scores without contamination value
    thresh = np.quantile(lof_scores, .03) #definotion of the threshold


    detected_outliers =  np.array([1] *df.shape[0])
    detected_outliers[lof_scores<=thresh]=-1
    
    return(y_pred,detected_outliers, thresh) #y_pred with fixed contamination, detected_out with a made thresh


    #possibility to visualize it with pca like the IF

In [ ]:
def dbscan_method(df):
    
    #encode categorical into multiple code if needed
    #df=multicolumnlabelencoder(df)
    #obj_df = df.select_dtypes(include=['object']).copy()
    #df=MultiColumnLabelEncoder(columns = obj_df.columns).fit_transform(df)
    
    #compute the distance thanks to KNN  to find eps
    neighbors = NearestNeighbors(n_neighbors=math.floor(math.sqrt(df.shape[0])))
    neighbors_fit = neighbors.fit(df)
    distances, indices = neighbors_fit.kneighbors(df)

    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    distancesbis=[0 for x in range(len(distances))]
    for i in range(len(distancesbis)-1):
        #print(i)
        distancesbis[i] = distances[i+1]
        i=i+1
    diff=distancesbis-distances

    thedistance=distances[np.where(diff>round(np.percentile(diff, 75)))] #hyper parameters
    try:
        eps_knn=thedistance[0]
    except IndexError:
        eps_knn=0.5
    

    # specify & fit model
    model = DBSCAN(eps = eps_knn, min_samples=50, metric='manhattan').fit(df) #parameters found with optuna
    # outliers dataframe
    outliers = df[model.labels_ == -1]
    y_pred=model.labels_ 
    y_pred[y_pred!=-1]=1 #because can have several classes, -1 is noise and could be others classes
    
    return(outliers,y_pred,eps_knn)

In [ ]:
def iqr_threshold_method(scores, margin):
    q1 = np.percentile(scores, 25, interpolation='midpoint')
    q3 = np.percentile(scores, 75, interpolation='midpoint')
    iqr = q3-q1
    lower_range = q1 - (1.5 * iqr)
    upper_range = q3 + (1.5 * iqr)
    lower_range = lower_range - margin
    upper_range = upper_range + margin
    return lower_range, upper_range

#cf description de la function dans outliers_research

def OdinOutlier(data,n_neighbors):
    margin=0
    radius=1.0
    algorithm='auto'
    leaf_size=30
    metric='minkowski'
    p=2
    metric_params=None
    n_jobs=None

    nn = neighbors.NearestNeighbors(n_neighbors=n_neighbors, radius=radius, algorithm=algorithm, leaf_size=leaf_size, metric=metric, p=p, metric_params=metric_params, n_jobs=n_jobs)
    #data=multicolumnlabelencoder(data)
    nn.fit(data)
    graph = nn.kneighbors_graph()
    indegree = graph.sum(axis=0)  # sparse matrix

    # smaller indegree means more of an anomaly
    # simple conversion to [0,1] so larger score is more of anomaly
    scores = (indegree.max() - indegree) / indegree.max()
    scores = np.array(scores)[0]

    lower_range, upper_range = iqr_threshold_method(scores, margin)

    anomaly_index = []
    for i in range(len(scores)):
        if scores[i] < lower_range or scores[i] > upper_range:
            scores[i]=-1
            anomaly_index.append(i)
        else:
            scores[i]=1

    return anomaly_index, scores

In [ ]:
def manu_uni_zscore(df, threshold,majority_perc, rare_perc): #    threshold=3, majority=0.95, rare=0.1

    import numpy as npa
    from scipy import stats
    from scipy.stats import zscore

    matrix_out=np.zeros([df.shape[0],df.shape[1]])
    detected_outliers=np.array([1] *df.shape[0])

    numeric_cols = df.select_dtypes(include=[np.number]).columns #'int','float', 'cfloat'


    #first option for numeric: z score
    #print(numeric_cols)
    if len(numeric_cols)>0: 
        data_zscore =df[numeric_cols].apply(stats.zscore)
        data_zscore = data_zscore.abs()

    #outliers preprocessing numeric
    for var in data_zscore.columns: #name of the var, here only numeric
        var_out=0
        for i in range(data_zscore.shape[0]):
        #print(i)
            if data_zscore.loc[i,var] > threshold:
                matrix_out[i, list(df.columns).index(var)]=1
                detected_outliers[i]=-1



    data_cat = df.select_dtypes(include=['object']).copy() #, "category

    #outliers preprocessing categoric
    for var in data_cat.columns: #name of the var, here only cat

        dict_var= pd.DataFrame.from_dict(data_cat[var].value_counts()).reset_index()
        dict_var=dict_var.rename(columns={'attribut': var, var:'count'})
        dict_var['count']=dict_var['count'] #/df.shape[0]
        #print(dict_var)

        # observation rare
        liste_rare=list(dict_var[(dict_var['count']==min(dict_var['count']))& (median(dict_var['count'])>=rare_perc*df.shape[0])]['index'])
        if len(liste_rare)!=0:
            where_errors=np.isin(df[var], liste_rare)

            matrix_out[where_errors, list(df.columns).index(var)]=1

        #ne correspond pas à la majorité 
        dict_count= pd.DataFrame.from_dict(dict_var['count'].value_counts()).reset_index()
        #print(dict_count)
        major=list(dict_count[dict_count['count']>majority_perc*df.shape[0]]['index'])
        if len(major)!=0:
            liste_rare=list(dict_var[dict_var['count']!=major[0]]['index'])
            where_errors=np.isin(df[var], liste_rare)
            matrix_out[where_errors, list(df.columns).index(var)]=1
            detected_outliers[where_errors]=-1

        
    return(matrix_out, detected_outliers)

In [ ]:
def manu_iqr(df,majority_perc, rare_perc):

    import numpy as np
    from scipy import stats
    from scipy.stats import zscore
    import seaborn as sns
    import statistics

    matrix_out=np.zeros([df.shape[0],df.shape[1]])
    detected_outliers=np.array([1] *df.shape[0])

    numeric_cols = df.select_dtypes(include=[np.number]).columns #'int','float', 'cfloat'


    #first option for numeric:IQR for outliers detection
    #print(numeric_cols)
    if len(numeric_cols)>0: 
        for var in numeric_cols: #name of the var, here only numeric
            data=df[var]
            sort_data = np.sort(df[var])
            Q1 = np.percentile(data, 25, interpolation = 'midpoint') 
            Q2 = np.percentile(data, 50, interpolation = 'midpoint') 
            Q3 = np.percentile(data, 75, interpolation = 'midpoint')
            IQR = Q3 - Q1 
            low_lim = Q1 - 1.5 * IQR
            up_lim = Q3 + 1.5 * IQR

            for i in range(len(data)):
                if ((data[i]> up_lim) or (data[i]<low_lim)):
                    matrix_out[i, list(df.columns).index(var)]=1
                    detected_outliers[i]=-1
                    


    data_cat = df.select_dtypes(include=['object']).copy() #, "category

    #outliers preprocessing categoric
    for var in data_cat.columns: #name of the var, here only cat

        dict_var= pd.DataFrame.from_dict(data_cat[var].value_counts()).reset_index()
        dict_var=dict_var.rename(columns={'attribut': var, var:'count'})
        dict_var['count']=dict_var['count'] #/df.shape[0]
        #print(dict_var)

        # observation rare c'est le minimum et la medianne a plus de 10% des observations (plusieurs catégories balanced)
        liste_rare=list(dict_var[(dict_var['count']==min(dict_var['count']))& (median(dict_var['count'])>=rare_perc*df.shape[0])]['index'])
        if len(liste_rare)!=0:
            where_errors=np.isin(df[var], liste_rare)

            matrix_out[where_errors, list(df.columns).index(var)]=1

        #ne correspond pas à la majorité qui a plus de 0.95 des données
        dict_count= pd.DataFrame.from_dict(dict_var['count'].value_counts()).reset_index()
        #print(dict_count)
        major=list(dict_count[dict_count['count']>majority_perc*df.shape[0]]['index'])
        if len(major)!=0:
            liste_rare=list(dict_var[dict_var['count']!=major[0]]['index'])
            where_errors=np.isin(df[var], liste_rare)
            matrix_out[where_errors, list(df.columns).index(var)]=1
            detected_outliers[where_errors]=-1

    return(matrix_out, detected_outliers)

In [ ]:
def multi_zscore(df,threshold): #    threshold=3

    import numpy as npa
    from scipy import stats
    from scipy.stats import zscore


    detected_outliers=np.array([1] *df.shape[0])

    numeric_cols = df.columns #'int','float', 'cfloat'


    #first option for numeric: z score
    #print(numeric_cols)
    if len(numeric_cols)>0: 
        data_zscore =df[numeric_cols].apply(stats.zscore)
        data_zscore = data_zscore.abs()

    #outliers preprocessing numeric
    for var in data_zscore.columns: #name of the var, here only numeric
        var_out=0
        for i in range(data_zscore.shape[0]):
        #print(i)
            if data_zscore.loc[i,var] > threshold:
                detected_outliers[i]=-1


        
    return(detected_outliers)

In [ ]:
def multi_iqr(df):

    import numpy as np
    from scipy import stats
    from scipy.stats import zscore
    import seaborn as sns
    import statistics

    detected_outliers=np.array([1] *df.shape[0])
    numeric_cols = df.columns #'int','float', 'cfloat'


    #first option for numeric:IQR for outliers detection
    #print(numeric_cols)
    if len(numeric_cols)>0: 
        for var in numeric_cols: #name of the var, here only numeric
            data=df[var]
            sort_data = np.sort(df[var])
            Q1 = np.percentile(data, 25, interpolation = 'midpoint') 
            Q2 = np.percentile(data, 50, interpolation = 'midpoint') 
            Q3 = np.percentile(data, 75, interpolation = 'midpoint')
            IQR = Q3 - Q1 
            low_lim = Q1 - 1.5 * IQR
            up_lim = Q3 + 1.5 * IQR

            for i in range(len(data)):
                if ((data[i]> up_lim) or (data[i]<low_lim)):
                      detected_outliers[i]=-1


    return(detected_outliers)

In [ ]:
def imputation_nainf(df): #pd dataframe
    df.replace([np.inf, -np.inf], np.nan, inplace=True) #replace inf by nan
    
    #the nan are replaced by a simple univariate feature imputation
    from sklearn.impute import SimpleImputer
    imp = SimpleImputer(strategy="most_frequent") #easier with the paramters
    #most frequent so it can work for both numeric and categoric without affecting the fact of being outliers. 
    
 
    return imp.fit_transform(df)

def support(pattern,liste_patterns):#frequency of an occurring pattern d
    #print(d_i,pattern)
    
    #n is the number of records in the dataset
    #d in D: categorical pattern where attributes occur only once
    #xi(D): cetegorical values of point xi for attributes D
    
    support=0 #initialisation
    
    d=len(pattern)
    #print(d)
    for liste_pattern in liste_patterns: #size the depth of the tree
        if pattern==liste_pattern[:d]:
            support=support+1
    
    if support==0:
        print('error support=0, pattern not in the dataframe')
        support=1
    
    return support #support of the pattern_Xi

def lda_function(df): #least discriminative attribute
    L_a=[]
    new_order=[]
    for variable in df.columns:
        L_a.append(df[variable].nunique())
    L_a=sorted(L_a, reverse=True)

    while L_a[0]==df.shape[0]:
        L_a.append(L_a.pop(0))

    for nbr_unique in L_a:
        for var in df.columns:
            if (df[var].nunique()==nbr_unique):
                new_order.append(var)
    new_order

    return new_order
def LDA_tree(df,new_order):
    #print(new_order)
    
    df=df[new_order]
    
    liste_patterns=df.values.tolist() #each observation is converted in a liste
    
    return liste_patterns

def score1(pattern: list,liste_patterns:list, threshold): #pattern of x_i
    score_out=0
    d=len(pattern)
    if abs(d)>0:
        for d_i in range(1,d+1):
            #print('d_i',d_i)
            if support(pattern[:d_i],liste_patterns) > threshold:
                score_out=score_out+ (1/(support(pattern[:d_i],liste_patterns)*abs(d_i)))
        #abs(d) is the depth of the tree
        #d represents the minimum infrequent pattern of the LDA-tree
    else: 
        print('Erreur d negative')
    return score_out #if close to 1, xi high proba of being outlier
#algo 1: attribute bagging algorithm
#randomly create pultiple subsets of the data to handle irrevelant and redundant attributes

from random import sample 

def attribute_bagging(df,T):
    #D_t attribute subset 
    #b = maximum nbr of attributes in attribute bag 
    subsets=None
    for t in (1,T): #T iteration 
        #take a random sample of the attributes without replacement 
        selected_attributes=df.sample(df.shape[1]) 
        b=max(selected_attributes.shape[1])
        #d in b attributes in the t th iteration
        subsets= selected_attributes #add selected attributes of D
        
    return subsets

#algo 2 postponed outlier detection in the paper, adapted

def postponed_outlier_detection(df,sigma): #for categorical attributes
    #input D dataset and sigma percentage (threshold)
    
    obj_df = df.select_dtypes(include=['object']).copy()
    df_cat=obj_df
    liste_var_cat= list(obj_df.columns)
    #print(liste_var_cat)
    m_c=len(liste_var_cat) #nbr of catagorical attributes
    if m_c!=0: #there are categorical variables
        threshold=sigma*df.shape[0]


        X_score1=[]
        X_freq=[] #vector containing all records
        X_i=[]
        X_infrequent=[]


        new_order=lda_function(df_cat)
        #print(lda)
        #print(variable_first)
        liste_patterns= LDA_tree(df_cat,new_order)
        #print(liste_patterns)
        for i in range(df_cat.shape[0]):
            pattern=liste_patterns[i]
            #support_i=support(d_i,pattern,liste_patterns)
            X_freq.append(pattern)
            X_i.append(i)
            #print(pattern)
            #print(type(pattern))
            #print(liste_patterns)
            score_out=score1(pattern,liste_patterns, threshold)
            support_i=support(pattern,liste_patterns)
            #print(score_out)
            X_score1.append(score_out) #to be reshape in 2 dimension i and score, and to be ranked in a decreasing order
            if support_i <=threshold:
                X_infrequent.append(i)
    else:
        X_score1=[]
        X_freq=[]
        X_infrequent=[]
        print('No categorical variables')

        
    return(X_score1,X_freq,X_infrequent)

def score2(df):
    import pyclustering
    from pyclustering.cluster.kmedians import kmedians
    from sklearn.metrics import silhouette_samples, silhouette_score

    N=df.shape[0]
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num=df.select_dtypes(include=numerics)
    liste_var_num=list(df_num.columns) #list of numerical variables
    
    sigma=0.05 #TO FIX
    threshold=int(sigma*N)
    
    if len(liste_var_num)!=0: #si il y a des variables numériques

        range_n_clusters = [2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
        silouhette_list=[]


        #define M
        for n_clusters in range_n_clusters:

            initial_medians=df_num.sample(n=n_clusters)

            # Initialize the clusterer with n_clusters value and a random generator
            # seed of 10 for reproducibility.
            kmedians_instance = kmedians(df_num, initial_medians)
            kmedians_instance.process()
            cluster_labels = kmedians_instance.get_clusters()
            medians = kmedians_instance.get_medians()

            labels=np.zeros(df_num.shape[0])
            for i in range(len(cluster_labels)):
                labels[cluster_labels[i]]=i+1
            #print(labels)

            # The silhouette_score gives the average value for all the samples.
            # This gives a perspective into the density and separation of the formed
            # clusters
            silhouette_avg = silhouette_score(df_num, labels)
            silouhette_list.append( silhouette_avg )
            #print("For n_clusters =", n_clusters,
                 # "The average silhouette_score is :", silhouette_avg)

            minimum=np.min(silouhette_list)
            #print(minimum)
            M=np.where(silouhette_list==minimum)[0][0]+2 #decalage python + 1 et + 1 de cluster pas commençant à 1

        #print('The selected M is', M,'.' ) #M=3 #(nbr of classes we want + outliers)

        #Kmedians most rebust than kmeans


        #initialization with k points (k number choosen as the nbr of clusters ) and
        #with d cordoniate: the number of numeric variables
        initial_medians=[[0 for col in range(len(liste_var_num))] for row in range(M)] #fixed them empty this is easier
        kmedians_instance = kmedians(df_num, initial_medians) #included the number of clusters
        kmedians_instance.process()
        cluster_labels = kmedians_instance.get_clusters()
        medians = kmedians_instance.get_medians() #Medians are calculated instead of centroids.
        #The algorithm is less sensitive to outliers than K-Means.

        #score 2

        X_score2=[]
        for i in range(df_num.shape[0]):
            score2=0
            m_q= df_num.shape[1] #number of continuous attributes
            for j in range(abs(m_q)):
                score2=score2+abs(df_num.iloc[i,j] -medians[0][j]) #manhattan distance
            X_score2.append(score2)

        #if score2 increases, the closest centroid for record x_i is distant
    else: #no variables numeriques
        M=None
        X_score2=[]
    
    return(M,X_score2)

def final_score(df,M,X_score1,X_freq,X_infrequent,X_score2):
    N=df.shape[0] #nbr d'observation x_i
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    obj_df = df.select_dtypes(include=['object']).copy()
    liste_var_cat= obj_df.columns #liste des variables categoriques
    num_df=df.select_dtypes(include=numerics)
    liste_var_num=num_df.columns #list of numerical variables

    score_rank=pd.DataFrame(columns=['df_index','score_cat','rank_cat','score_num','rank_num','final_rank'])

    for i in range(N):
        #compute for each attribute the scores
        score_rank.loc[i,'df_index']=i
        if len(X_score1)!=0: #not empty
            score_rank.loc[i,'score_cat']=X_score1[i]
        if len(X_score2)!=0: #not empty
            score_rank.loc[i,'score_num']=X_score2[i]
    score_meltzer=score_rank


    #sort the individues by their scores for each attributes    
    rank_cat=score_rank.sort_values('score_cat')
    rank_cat=rank_cat.reset_index()
    rank_num=score_rank.sort_values('score_num')
    #print(rank_num)
    rank_num=rank_num.reset_index()
    #print(rank_num)

    for i in range(N):
        #print(rank_cat[rank_cat['df_index']==i])
        score_meltzer.loc[i,'rank_cat']=rank_cat[rank_cat['df_index']==i].index[0]
        score_meltzer.loc[i,'rank_num']=rank_num[rank_num['df_index']==i].index[0]
        #add the rank for each attribute into the dataset

    score_meltzer['final_rank']=(score_meltzer['rank_cat']+score_meltzer['rank_num'])/2


    return(score_meltzer) #return the dataset to have all the scores and check them if needed but decision on final_rank
        

def meltzer_algo(df,sigma): #sigma=0.05
    import signal
    X_score1,X_freq,X_infrequent=postponed_outlier_detection(df,sigma)
    M,X_score2=score2(df)
    score_meltzer=final_score(df,M,X_score1,X_freq,X_infrequent,X_score2)
    score_meltzer=score_meltzer.sort_values('final_rank')
    score_meltzer= score_meltzer.reset_index()

    thres=int(sigma*df.shape[0])
    #print(thres)

    outliers_metzler=np.array([1] *df.shape[0])
    for i in range(len(score_meltzer.loc[:thres,'df_index'].values)):
        outliers_metzler[score_meltzer.loc[:thres,'df_index'].values[i]]=-1
    return outliers_metzler,M
    
    

In [ ]:
def factorielle(n):
    n=int(n)
    if n == 0:
        return 1
    else:
        F = 1
        
        for k in range(2,n+1):
            F = F * int(k)
        return F
    
def fct_toknow_the_type(df,var):
    nbr=df[df.columns[var]].nunique()
    
    if nbr==df.shape[0]:
        return 'id'
        
    if nbr==2:
        return 'binary'
    from pandas.api.types import is_string_dtype
    
    if is_string_dtype(df[df.columns[var]])==True:
        return 'nominal'
    
    from scipy.stats import shapiro
    #drawback of Shapiro-Wilk test is,
    #it is not reliable once the sample size (or) the length of a variable crosses 5,000.
    
    if shapiro(df[df.columns[var]])[1]>0.05:
        
        return 'gaussian'

    else:
        return 'count'
    


def Mixed_RBM_Simplified(df, a, Beta): #a=[1]*df.shape[0], Beta=0.05
    
    matrix_nrj=np.zeros(df.shape)

    for var in range(1,df.shape[1]):
        #print(var)
            #energy fct
        if fct_toknow_the_type(df,var)=='id':
            #print('id')
            matrix_nrj[:][var]= 1 #to have exp of 1 equals to zero

        if fct_toknow_the_type(df,var)=='binary':
            #print('binary')
            for obs in range(df.shape[0]):
                if obs==0:
                    value=df.iloc[obs,var]
                if df.iloc[obs,var]==value:
                    df.iloc[obs,var]=0
                else:
                    df.iloc[obs,var]=1
                #print(obs)
                #print(df.iloc[obs,var], 'binary')
                matrix_nrj[obs][var]= - a[obs]*df.iloc[obs,var]

        if fct_toknow_the_type(df,var)=='gaussian':
            #print('gaussian')
            for obs in range(df.shape[0]):
                #print(df.iloc[obs,var], 'gaussian')
                matrix_nrj[obs][var]=0.5*(df.iloc[obs,var]**2) - a[obs]*df.iloc[obs,var]

        if fct_toknow_the_type(df,var)=='nominal':#print('nominal')
            for obs in range(df.shape[0]):
                matrix_nrj[obs][var]=list(df[df.columns[var]]).count(df.iloc[obs,var])

        if fct_toknow_the_type(df,var)=='count':
            #print('count')
            for obs in range(df.shape[0]):
                #print(obs)
                if df.iloc[obs,var]>10:
                    fact=float(10000) #sinon trop gros
                else:
                    fact=float(factorielle(df.iloc[obs,var])) #limite du factoriel for acceptance
                matrix_nrj[obs][var]= np.log(fact) - a[obs]*df.iloc[obs,var]

        outliers=np.array([1]*df.shape[0])
        score_nrj_list=[]
        for nb_obs in range(df.shape[0]):
            score_nrj=-np.log(abs(np.sum(matrix_nrj[nb_obs])))
            score_nrj_list.append(score_nrj)
            #print(score_nrj)
        sort_nrj_list=sorted(score_nrj_list)
        Beta_values=[]

        for nbr in range(int(Beta*df.shape[0])):
            #print(nbr)
            Beta_values.append(sort_nrj_list[nbr])
            Beta_values.append(sort_nrj_list[-nbr] )


        for nbr_obs in range(df.shape[0]):

            if (score_nrj_list[nbr_obs] in Beta_values):
                outliers[nbr_obs]=-1
    return(outliers)
    

### How to call the functions? 

In [ ]:
#import your dataset
df=pd.read_csv(path + name + '.csv', sep=',') #df with outliers

In [ ]:
#multi label encoding if needed(mixed and categorical values)
obj_df = df.select_dtypes(include=['object', 'bool']).copy()
df=MultiColumnLabelEncoder(columns = obj_df.columns).fit_transform(df)

In [ ]:
#isolation forest multivariate detection, need encoding

import time

start = time.time()
df_if=isolationforrest_multi(df) #,  threshold)
#acc=accuracy_score(know_outliers,np.array(df_if['anomaly_label'].values))
#f1=f1_score(know_outliers,np.array(df_if['anomaly_label'].values), average=None) #df_if['label_outliers']
#acc, f1=metrics(know_outliers,np.array(df_if['label_outliers'].values)) #df_if['label_outliers'])) #auc and f1 score
#matrix= confusion_matrix(know_outliers,np.array(df_if['anomaly_label'].values))#df_if['label_outliers'])
outliers_vector=df_if['anomaly_label']
(time.time() - start)

In [ ]:
#Lof by Beunig detection, need encoding

n_neighbors=28
contamination=0.15
detected_outliers,thesh=LOF_b(df,n_neighbors,contamination)
acc=accuracy_score(know_outliers,detected_outliers)


#DBSCAN, need encoding


outliers,found_labels,eps_knn=dbscan_method(df)
found_labels[found_labels!=-1]=1
detected_outliers=found_labels

#odin, need encoding
n_neighbors=28
anomaly_index,detected_outliers= OdinOutlier(np.array(df),n_neighbors)

#Zscore need encoding
threshold=3
detected_outliers= multi_zscore(df,threshold)

#IQR need encoding or just numeric
detected_outliers=multi_iqr(df)


Without encoding the categorical values:

In [ ]:
#ALGO: Meltzer-------------------------------------------------------------------------------------------------
sigma=0.05

detected_outliers,M=meltzer_algo(df,sigma)

#ALGO: RBM_simplied-------------------------------------------------------------------------------------------------
Beta=0.05
a=[1]*df.shape[0]
detected_outliers=Mixed_RBM_Simplified(df, a, Beta)

#matrix for univariate and vector for multivariate

#ALGO: Zscore + ICL-------------------------------------------------------------------------------------------------
threshold=2
majority_perc=0.95
rare_perc=0.19
matrix_out,detected_outliers=manu_uni_zscore(df,threshold,majority_perc,rare_perc)

#ALGO: IQR+ ICL-------------------------------------------------------------------------------------------------
majority_perc=0.95
rare_perc=0.19
matrix_out,detected_outliers=manu_iqr(df,majority_perc,rare_perc)


In [ ]:
#for univariate

#isolation forests
iforest_matrix=isolationforrest_uni(df_encode)
TP, FN, FP,TN, acc, f1=confusion_matrix_for_matrix(matrix_out= matrix_out, algo_matrix=matrix_pred) #matrix True or false if founded

